In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score

import pickle as pkl

#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter(log_dir="/content/drive/MyDrive/Colab Notebooks/tensorboardBN")

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else{}
print(torch.cuda.is_available()) 

True


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# Download data

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32,padding=4), transforms.RandomHorizontalFlip(), transforms.ToTensor()])
training_data = datasets.CIFAR100(root='.data', train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR100(root='.data', train=False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_set, val_set = torch.utils.data.random_split(training_data,[40000,10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, drop_last=True, **LOADER_KWARGS)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=128, shuffle=True, drop_last=True, ** LOADER_KWARGS)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=True, drop_last=True, **LOADER_KWARGS)

In [ ]:
training_loader = torch.utils.data.DataLoader(training_data, batch_size=128, shuffle=True, drop_last=True, **LOADER_KWARGS)

In [ ]:
print(training_loader.dataset)

Dataset CIFAR100
    Number of datapoints: 50000
    Root location: .data
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )


In [ ]:
with open("/content/drive/MyDrive/CIFAR/training_loader100.pt", "wb") as f:
  torch.save(training_loader, f)

with open("/content/drive/MyDrive/CIFAR/test_loader100.pt", "wb") as f:
  torch.save(test_loader, f)


#OOD dataset: SVHN

In [ ]:
svhn_dataset = datasets.SVHN(root='.data', split='test', transform=transforms.ToTensor(), download=True)
svhn_loader = torch.utils.data.DataLoader(svhn_dataset, batch_size=128, drop_last=True, **LOADER_KWARGS)

  0%|          | 0/64275384 [00:00<?, ?it/s]

# Loading data in drive

In [ ]:
with open("/content/drive/MyDrive/CIFAR/training_loader100.pt", "rb") as f:
  training_loader = torch.load(f)

with open("/content/drive/MyDrive/CIFAR/test_loader100.pt", "rb") as f:
  test_loader = torch.load(f)

In [ ]:
print(test_loader.dataset)

Dataset CIFAR100
    Number of datapoints: 10000
    Root location: .data
    Split: Test
    StandardTransform
Transform: ToTensor()


# Network

In [ ]:
eps = 1e-20

In [ ]:
class Gaussian:
  def __init__(self, mu, rho):
    self.mu = mu
    self.rho = rho
    self.normal = torch.distributions.Normal(0,1)
  
  @property
  def sigma(self):
    return torch.log1p(torch.exp(self.rho))
  
  def sample(self):
    epsilon = self.normal.sample(self.rho.size()).to(DEVICE)
    return self.mu + self.sigma * epsilon
  
  def log_prob(self, input):
    return (-math.log(math.sqrt(2 * math.pi)) - torch.log(self.sigma+eps) - ((input - self.mu) ** 2) / (2 * self.sigma ** 2)).sum()

In [ ]:
class GaussianPrior:
  def __init__(self,mu,sigma):
    self.mu = mu
    self.sigma = sigma
  
  def log_prob(self,input):
    return (-math.log(math.sqrt(2 * math.pi)) - torch.log(self.sigma) - ((input - self.mu) ** 2) / (2 * self.sigma ** 2)).sum()

In [ ]:
class myLinear(nn.Module):
  def __init__(self, n_input, n_output, sigma1):
    super().__init__()
    self.n_input = n_input
    self.n_output = n_output
    

    self.w_mu = nn.Parameter(torch.Tensor(n_output,n_input).normal_(0,math.sqrt(2/n_input)))
    

    self.b_mu = nn.Parameter(torch.Tensor(n_output).normal_(0,math.sqrt(2/n_input)))
    
  def forward(self, input, sample=False):
    
    w = self.w_mu
    b = self.b_mu
    
  
    return F.linear(input, w, b)


In [ ]:
class myConv2D(nn.Module):
  def __init__(self, in_channels, out_channels, sigma1, kernel_size=3, stride=1, padding=1):
    super().__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding

    self.w_mu = nn.Parameter(torch.Tensor(out_channels,in_channels, kernel_size, kernel_size))
    self.reset_para()
  
  def reset_para(self):
    nn.init.kaiming_uniform_(self.w_mu, a=math.sqrt(5))
  
  def forward(self, input, sample=False):
    
    w = self.w_mu
    
    
    return F.conv2d(input, w, bias=None, stride=self.stride, padding=self.padding)

In [ ]:
def myConv3x3(in_channels, out_channels, sigma1, stride=1):
  return myConv2D(in_channels, out_channels, sigma1, kernel_size=3,stride=stride, padding=1)


In [ ]:
class TLU(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.num_features = num_features
    self.tau = nn.parameter.Parameter(torch.Tensor(1,num_features,1,1), requires_grad=True)
    self.reset_parameters()
  
  def reset_parameters(self):
    nn.init.kaiming_normal_(self.tau)
    
  def forward(self, x):
    return torch.max(x, self.tau)

  

In [ ]:
class FRN(nn.Module):
  def __init__(self, num_features, eps=1e-6, is_eps_learnable=False):
    super().__init__()
    self.num_features = num_features
    self.init_eps = eps
    self.is_eps_learnable = is_eps_learnable

    self.weight = nn.parameter.Parameter(torch.Tensor(1, num_features, 1, 1), requires_grad=True)
    self.bias = nn.parameter.Parameter(torch.Tensor(1,num_features, 1, 1), requires_grad=True)
    if is_eps_learnable:
      self.eps = nn.Parameter(torch.Tensor(1))
    else:
      self.eps = torch.tensor(eps)
    self.reset_parameters()
  
  def reset_parameters(self):
    nn.init.kaiming_normal_(self.weight)
    nn.init.kaiming_normal_(self.bias)
    if self.is_eps_learnable:
      nn.init.constant_(self.eps, self.init_eps)

  def forward(self,x):
    nu2 = x.pow(2).mean(dim=[2,3], keepdim=True)

    x = x * torch.rsqrt(nu2 + self.eps.abs())
    x = self.weight * x + self.bias
    return x

In [ ]:
class myResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, sigma1, stride=1, downsample=None):
    super().__init__()
    self.conv1 = myConv3x3(in_channels, out_channels, sigma1, stride)
    self.frn1 = nn.BatchNorm2d(out_channels)
    self.tlu1 = nn.ReLU(inplace=True)
    self.conv2 = myConv3x3(out_channels, out_channels, sigma1)
    self.frn2 = nn.BatchNorm2d(out_channels)
    self.tlu2 = nn.ReLU(inplace=True)
    self.downsample = downsample
  
  def forward(self, x):
    residual = x
    out = self.conv1(x)
    out = self.frn1(out)
    out = self.tlu1(out)
    out = self.conv2(out)
    out = self.frn2(out)
    if self.downsample:
      residual = self.downsample(x)
    out += residual
    out = self.tlu2(out)
    return out
  


In [ ]:
class myResNet14(nn.Module):
  def __init__(self, sigma1, num_class=10):
    super().__init__()
    self.in_channels = 16
    self.conv = myConv3x3(3,16, sigma1)
    self.frn = nn.BatchNorm2d(16)
    self.tlu = nn.ReLU(inplace=True)

    self.block1 = myResidualBlock(16,16,sigma1)
    self.block2 = myResidualBlock(16,16,sigma1)

    downsample1 = nn.Sequential(myConv3x3(16,32,sigma1,2), nn.BatchNorm2d(32))
    self.block3 = myResidualBlock(16,32,sigma1,2,downsample1)
    self.block4 = myResidualBlock(32,32,sigma1)

    downsample2 = nn.Sequential(myConv3x3(32,64,sigma1,2), nn.BatchNorm2d(64))
    self.block5 = myResidualBlock(32,64,sigma1,2,downsample2)
    self.block6 = myResidualBlock(64,64,sigma1)

    self.avg_pool = nn.AvgPool2d(8)
    self.fc = myLinear(64, num_class, sigma1)

  def forward(self, x, sample=False):
    out = self.conv(x)
    out = self.frn(out)
    out = self.tlu(out)
    out = self.block1(out)
    out = self.block2(out)
    out = self.block3(out)
    out = self.block4(out)
    out = self.block5(out)
    out = self.block6(out)
    out = self.avg_pool(out)
    out = out.view(out.size(0),-1)
    out = F.softmax(self.fc(out, sample))
    return out
  
   
  
  def free_energy(self, input, target, batch_size, num_batches, n_samples, T):
    negative_log_likelihood = torch.zeros(1).to(DEVICE)
    for i in range(n_samples):
      output = self(input, sample=True)
      negative_log_likelihood += F.nll_loss(torch.log(output+eps), target, size_average=False)/n_samples

    # new target function, not absorb T into prior
    loss = negative_log_likelihood / T * num_batches 

    corrects = output.argmax(dim=1).eq(target).sum().item()

    return loss, corrects

  
  

In [ ]:
def write_weight_histograms(epoch):
  writer.add_histogram('histogram/w1_mu', net.l1.w_mu, epoch)
  writer.add_histogram('histogram/w1_rho', net.l1.w_rho, epoch)
  writer.add_histogram('histogram/w2_mu', net.l2.w_mu, epoch)
  writer.add_histogram('histogram/w2_rho', net.l2.w_rho, epoch)
  writer.add_histogram('histogram/w3_mu', net.l3.w_mu, epoch)
  writer.add_histogram('histogram/w3_rho', net.l3.w_rho, epoch)

def write_loss_scalars(epoch, loss, accuracy):
  writer.add_scalar('logs/loss', loss, epoch)
  writer.add_scalar('logs/accuracy', accuracy, epoch)

def write_test_scalar(epoch, loss, accuracy):
  writer.add_scalar('logs/test_loss', loss,epoch)
  writer.add_scalar('logs/test_accuracy', accuracy, epoch)

def write_sigma(epoch):
  writer.add_scalar('block1/sigma', net.block1.sigma_mean,epoch)
  writer.add_scalar('block2/sigma', net.block2.sigma_mean,epoch)
  writer.add_scalar('block3/sigma', net.block3.sigma_mean,epoch)
  writer.add_scalar('block4/sigma', net.block4.sigma_mean,epoch)
  writer.add_scalar('block5/sigma', net.block5.sigma_mean,epoch)
  writer.add_scalar('block6/sigma', net.block6.sigma_mean,epoch)
  writer.add_scalar('fc/sigma', net.fc.sigma_mean,epoch)


# Train and test


In [ ]:
def update_lr(optimizer,lr):
  for param_group in optimizer.param_groups:
    param_group['lr']= lr

def train(net, optimizer, epoch, trainLoader, batchSize, nSamples ,T):
  net.train()
  num_batches_train = len(trainLoader)
  
 # if epoch == 0:
  #  write_weight_histograms(epoch)
  for batch_idx, (data, target) in enumerate(tqdm(trainLoader)):
    data, target = data.to(DEVICE), target.to(DEVICE)
    
    net.zero_grad()
    loss, corrects = net.free_energy(data, target, batchSize, num_batches_train, nSamples,T)
    loss.backward()
    optimizer.step()

    accuracy = corrects / batchSize
  #write_loss_scalars(epoch, loss, accuracy)
 # write_weight_histograms(epoch)
 # write_sigma(epoch)
  
  return accuracy, loss

In [ ]:
def test_duringTrain(net, epoch, testLoader, batchSize, nSamples, T, num_class=10):
  net.eval()
  accuracy = 0
  n_corrects = 0
  Loss = 0
  num_batches_test = len(testLoader)
  n_test = batchSize * num_batches_test
  outputs = torch.zeros(n_test, num_class).to(DEVICE)
  correct = torch.zeros(n_test).to(DEVICE)

  
  with torch.no_grad():
    for i, (data, target) in enumerate(testLoader):
      data, target = data.to(DEVICE), target.to(DEVICE)
      for j in range(nSamples):
        output = net(data, sample=True)
        outputs[i*batchSize:batchSize*(i+1), :] += output/nSamples
        Loss +=  F.nll_loss(torch.log(output), target, size_average=False)/nSamples
        # loss is log likelihood
        if j == nSamples - 1:
          correct[i*batchSize:batchSize*(i+1)] = (outputs[i*batchSize:batchSize*(i+1), :]).argmax(1).eq(target)
        
    accuracy = correct.mean()
   # write_test_scalar(epoch, Loss, accuracy)
    
  return accuracy, Loss

In [ ]:
def test(net, testLoader, batchSize, nSamples,T):
  # update ECE
  net.eval()
  accuracy = 0
  n_corrects = 0
  Loss = 0
  num_batches_test = len(testLoader)
  n_test = batchSize * num_batches_test
  outputs = torch.zeros(n_test, 100).to(DEVICE)
  correct = torch.zeros(n_test).to(DEVICE)
  target_all = torch.zeros(n_test).to(DEVICE)
  
  M = 10
  boundary = ((torch.tensor(range(0,M))+1)/10).view(1,-1)
  boundary = boundary.repeat(batchSize, 1).to(DEVICE)
  
  acc_Bm_sum = torch.zeros(M).to(DEVICE)
  conf_Bm_sum = torch.zeros(M).to(DEVICE)
  Bm = torch.zeros(M).to(DEVICE)
  
  with torch.no_grad():
    for i, (data, target) in enumerate(testLoader):
      data, target = data.to(DEVICE), target.to(DEVICE)
      target_all[i*batchSize:batchSize*(i+1)] = target
      for j in range(nSamples):
        output = net(data, sample=True)
        outputs[i*batchSize:batchSize*(i+1), :] += output/nSamples
        Loss +=  F.nll_loss(torch.log(output), target, size_average=False)/nSamples
        # loss is log likelihood
        
      correct[i*batchSize:batchSize*(i+1)] = (outputs[i*batchSize:batchSize*(i+1), :]).argmax(1).eq(target)
      
      otemp =outputs[i*batchSize:batchSize*(i+1), :]
      p_i,_ = otemp.max(dim=1, keepdims=True)
      B = (p_i.le(boundary)*1).argmax(dim=1)
          
      acc_i = otemp.argmax(1).eq(target)
      for m in range(M):
        is_m = B.eq(m)
        Bm[m] += is_m.sum()
        acc_Bm_sum[m] += torch.sum(acc_i * is_m)
        conf_Bm_sum[m] += torch.sum(p_i.flatten() * is_m)

    accuracy = correct.mean()

  ROCAUC = roc_auc_score(target_all.cpu(), outputs.cpu(), multi_class='ovr')
  
  ECE = (acc_Bm_sum - conf_Bm_sum).abs().sum()/(n_test)

  temp = (acc_Bm_sum - conf_Bm_sum)/Bm
  temp[temp!=temp]=0
  MCE,_ = temp.abs().max(0)

  return accuracy, Loss, ECE, MCE, ROCAUC, output

In [ ]:
def cal_entropy(p):
  logP = p.clone()
  logP[p==0]=1
  logP = torch.log(logP)
  return (-logP*p).sum(dim=1)

def OOD_test(net, oodLoader, inDis_output, batchSize, nSamples, T, num_class=10):
  net.eval()
  num_batches_test = len(oodLoader)
  n_test = batchSize * num_batches_test
  n_inDis = len(inDis_output)

  outputs = torch.zeros(n_test, num_class).to(DEVICE)
  
  target_all = torch.zeros(n_test+n_inDis)
  target_all[n_test:] = 1

  score1 = torch.zeros(n_test+n_inDis)
  score2 = torch.zeros(n_test+n_inDis)

  with torch.no_grad():
    for i, (data, target) in enumerate(oodLoader):
      data = data.to(DEVICE)

      for j in range(nSamples):
        output = net(data,sample=True)
        outputs[i*batch_size:batchSize*(i+1), :] += output/nSamples
  entropy = cal_entropy(outputs)
  entropy_ave = entropy.mean()
  entropy_std = entropy.std()

  score1[:n_test],_ = outputs.max(dim=1)
  score1[n_test:],_ = inDis_output.max(dim=1)

  score2[:n_test] = entropy_ave
  score2[n_test:] = cal_entropy(inDis_output).mean()

  L2D  = (torch.square(outputs-0.1).sum(dim=1)).mean()
  ROCAUC1 = roc_auc_score(target_all, score1, multi_class='ovr', average='weighted')
  ROCAUC2 = roc_auc_score(target_all, score2, multi_class='ovr', average='weighted')
  return entropy_ave, entropy_std, L2D, ROCAUC1, ROCAUC2

# Basic setting of Network

In [ ]:
batch_size = 128

In [ ]:
# hyperparameter lists
T_list = torch.pow(10,-1*torch.tensor(range(0,45,5))/10).to(DEVICE)
sigma_list = torch.exp(-1*torch.tensor(range(0,5))).to(DEVICE)

n_samples = 1


# 3 pretrained network



In [ ]:
for i in range(3):
  T = T_list[0]
  sigma = sigma_list[0]
  net = myResNet14(sigma, num_class=100).to(DEVICE)
  max_lr = 0.001
  curr_lr = 0.001
  epochs = 300
  optimizer = optim.Adam(net.parameters(),lr=curr_lr)

  
  print(i)
  for epoch in range(epochs):
    trainAcc, trainLoss = train(net, optimizer, epoch+i*epochs, training_loader, batch_size, n_samples,T)
    curr_lr = max_lr/2 * (1+math.cos((epoch)/epochs*math.pi))
    update_lr(optimizer,curr_lr)
  with open(f"/content/drive/MyDrive/CIFAR/pretrainedNDA/pretrained100/net{i}.pt", "wb") as f:
    torch.save(net.state_dict(),f)
    
